## 1단계 : 필요한 모듈 임포트하기

In [27]:
import datetime

import numpy as np
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit

## 2단계 : 평가 함수(지니 계수) 정의
> Amount of probability of specific feature that is classified incorrectly when selected randomly

In [15]:
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    '''
    오름차순으로 정렬하기 위해 추출해야 하는 인덱스 순으로 정렬
    이 인덱스를 다시 y_true에 적용
    '''
    y_true = y_true[np.argsort(y_prob)]
    # 초기화
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    # ntrue, gini, delta 업데이트
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    # gini 계수
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

`@jit`을 사용해서 메인 구문 전후에 `jit` 함수가 데코레이터로 동작할 수 있게 합니다. 이는 컴파일을 최적화할 수 있는 역할을 한다고 하네요. 이어서 `gini_xgb`를 정의합니다.

In [16]:
def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = eval_gini(labels, preds)
    return[('gini', gini_score)]

## 3단계 : 타겟 인코딩

In [17]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


def target_encode(trn_series=None,    # Revised to encode validation series
                  val_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):

    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"]) # 학습 데이터의 각 카테고리별 계산
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

## 4단계 : 데이터 읽기

In [18]:
train_df = pd.read_csv('../input/train.csv', na_values="-1")
test_df = pd.read_csv('../input/test.csv', na_values="-1")


train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]

이중 상관관계가 강하게 나타나는 피쳐들을 `combs`라는 변수에 따로 저장합니다. 작성자에 따르면 몇가지 조합을 해 보았을 때, 이 조합을 선택했다고 하네요(). 주석으로 표시한 shadow 는 작성자의 feature selection module의 output 값이라고 합니다. ([링크](https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283/comments))

In [19]:
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),
    ('ps_reg_01', 'ps_car_04_cat'),
]

## 5단계 : 데이터 전처리

In [20]:
for (b,c) in enumerate(combs):
    print (b,c)

0 ('ps_reg_01', 'ps_car_02_cat')
1 ('ps_reg_01', 'ps_car_04_cat')


In [21]:
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in  enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)

X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

In [22]:
y_valid_pred = 0*y
y_test_pred = 0

## 6단계 : 모델 학습과 평가
폴드를 설정합니다. 여기에서는 KFold (fold = 5)를 사용하였습니다.

In [23]:
K = 5
kf = KFold(n_splits=K, random_state=1, shuffle=True)
np.random.seed(0)

In [24]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50

XGBoost Classifier 를 정의합니다. 우선, 각 파라미터의 의미는 다음과 같습니다.
- `n_estimators` : 결정 트리의 개수를 의미합니다. 많은 `n_estimator`를 추가하는 것은 과적합 우려가 있는데, 이는 Boosting 모델이 이전 트리의 에러를 수정하는 방식으로 새로운 트리를 만들어 나가기 때문입니다.
- `max_depth` : 트리의 깊이를 의미하며, 과적합을 조절하기 위해 사용합니다. 트리의 길이가 깊다면 모델은 특정한 샘플에 대한 매우 구체적인 관계를 학습할 수 있도록 하기 때문입니다.
- `objective` : 최소화되어야 하는 손실 함수를 의미합니다.
- `learning_rate` : 학습률을 의미합니다.
- `subsample` : 각 트리에서 선택되어야 할 관측의 일부분을 의미합니다. `subsample`가 작을수록 모델은 보수적이고 과적합을 방지하지만, 너무 작은 `subsample`은 과소적합될 우려가 있습니다. 일반적으로는 `0.5`와 `1` 사이의 값을 사용합니다.
- `min_child_weight` : 트리에서 새 노드를 만들기 위해서 필요한 최소한의 가중치를 의미합니다.
- `colsample_bytree` : 각 트리를 구성할 때 열의 하위 샘플링 비율입니다. 하위 샘플링은 트리가 구성될 때마다 한번씩만 발생합니다.
- `scale_pos_weight` : 불균형한 클래스에 유용한데요, 양수와 음수 가중치의 균형을 조정하기 위한 파라미터입니다. 여기서 사용하는 데이터셋이 불균형하기 때문에 이 파라미터를 잘 튜닝하는 것이 중요합니다.
- `gamma` : 트리의 리프 노드에서 추가 분하을 만들기 위해 필요한 최소한의 손실 감소를 의미합니다. `gamma` 값이 클수록 모델은 더욱 보수적일 것입니다.
- `reg_alpha` : 가중치에 가하는 L1 규제항을 의미합니다. `reg_alpha` 값이 클수록 모델은 더욱 보수적일 것입니다.
- `reg_lambda` : 가중치에 가하는 L2 규제항을 의미합니다. `reg_lambda` 값이 클수록 모델은 더욱 보수적일 것입니다.

In [25]:
model = XGBClassifier(
    n_estimators=MAX_ROUNDS,
    max_depth=4,
    objective="binary:logistic",
    learning_rate=LEARNING_RATE,
    subsample=.8,
    min_child_weight=6,
    colsample_bytree=.8,
    scale_pos_weight=1.6,
    gamma=10,
    reg_alpha=8,
    reg_lambda=1.3
)

교차 검증을 수행합니다.

In [28]:
# Run CV

for i, (train_index, test_index) in enumerate(kf.split(train_df)):

    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)

    # Enocode data
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # Run model for this fold
    if OPTIMIZE_ROUNDS:
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train,
                               eval_set=eval_set,
                               eval_metric=gini_xgb,
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS,
                               verbose=False
                             )
        print( "  Best N trees = ", model.best_ntree_limit )
        print( "  Best gini = ", model.best_score )
    else:
        fit_model = model.fit( X_train, y_train )

    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:,1]
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred

    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:,1]

    del X_test, X_train, X_valid, y_train

y_test_pred /= K  # Average test set predictions

print( "\nGini for full training set:" )
eval_gini(y, y_valid_pred)


Fold  0
[18:22:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.2851059728784705

Fold  1
[18:23:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
  Gini =  0.28185495483845957

Fold  2
[18:23:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore th

0.28501477642381845